In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

In [7]:
df = pd.read_csv("/content/Rainfall_Data_LL.csv")

In [8]:
df1 = df.loc[df['SUBDIVISION'] == 'Matathwada'].iloc[:, 2:16]
df2 = pd.melt(df1, id_vars='YEAR', value_vars=df1.columns[1:-1])
df2['Date'] = df2['variable'] + ' ' + df2['YEAR'].astype(str)
df2.loc[:,'Date'] = df2['Date'].apply(lambda x : datetime.strptime(x, '%b %Y'))
df2.columns=['Year', 'Month', 'Rainfall', 'Date']
df2.sort_values(by='Date', inplace=True)
df3 = df2.drop(columns = ["Month", "Year"])
df3.set_index("Date", inplace = True)

In [9]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00


In [10]:
from pmdarima import auto_arima
train = df3.iloc[:-90]
test = df3.iloc[-90:]

In [11]:
model3 = auto_arima(y = df3.Rainfall, m = 12)

In [13]:
predictions3 = pd.Series(model3.predict(n_periods = 12))
predictions3

2018-01-01      6.739256
2018-02-01      7.523164
2018-03-01     17.548508
2018-04-01     11.135269
2018-05-01     12.122091
2018-06-01    115.128523
2018-07-01    155.863195
2018-08-01    156.090233
2018-09-01    153.294718
2018-10-01     62.465946
2018-11-01     14.172963
2018-12-01      7.306057
Freq: MS, dtype: float64

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
pip install bz2file

  Preparing metadata (setup.py) ... done
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6868 sha256=f2ee488e83165347ad9b17377096edd9856a072227b7ac5c012a2c1d814f5cee
  Stored in directory: /root/.cache/pip/wheels/69/ee/f7/6fccd10cb65421ba2da64fa6caf8ee7fbae0059884af8c8587
Successfully built bz2file


In [16]:
import bz2file as bz2
import pickle

In [17]:
def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2', 'w') as f:
    pickle.dump(data, f)

In [18]:
compressed_pickle('model3', 'model3')

In [19]:
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = pickle.load(data)
  return data

In [20]:
model = decompress_pickle('model3.pbz2')

In [21]:
model3.predict(n_periods = 12)

2018-01-01      6.739256
2018-02-01      7.523164
2018-03-01     17.548508
2018-04-01     11.135269
2018-05-01     12.122091
2018-06-01    115.128523
2018-07-01    155.863195
2018-08-01    156.090233
2018-09-01    153.294718
2018-10-01     62.465946
2018-11-01     14.172963
2018-12-01      7.306057
Freq: MS, dtype: float64